In [12]:
from sklearn.neural_network import MLPClassifier
import numpy as np
import pickle
from tqdm import tqdm 

In [125]:
def analyse_CM(CM, toprint=False):
    [[TP, FP], [FN, TN]] = CM
    TPR = TP/(TP+FN) if (TP +FN) !=0 else 0
    TNR = TN/(TN+FP) if (FP +TN) !=0 else 0
    ACC = (TP+TN)/(TP+TN+FP+FN) if (TP+TN+FP+FN) !=0 else 0
    Precision = TP/(TP+FP) if (TP+FP) !=0 else 0
    FS = 2*TP/(2*TP+FP+FN) if (2*TP+FP+FN) !=0 else 0
    if toprint:
        print("TPR: ",TPR)
        print("TNR: ", TNR)
        print("ACC: ", ACC)
        print("Precision: ", Precision)
        print("FS: ",FS)
    return [TPR,TNR, ACC,Precision,FS]

In [148]:
with open('data/train_fallen.pk', 'rb') as f:
    train_samples = pickle.load(f)
obs = np.array(train_samples['obs'])
controller = np.array(train_samples['controller'])
t0 = train_samples['t0']
X, y = [], []
for i in tqdm(range(len(obs))):
    X.append(np.concatenate((obs[i,0,:28], obs[i,0,30:31], controller[i], [(t0[i]%20)/20])))
    y.append(obs[i][-1][30]>0.13)

100%|██████████| 97483/97483 [00:00<00:00, 217178.30it/s]


In [149]:
with open('data/eval_fallen.pk', 'rb') as f:
    samples = pickle.load(f)
obs = np.array(samples['obs'])
controller = np.array(samples['controller'])
t0 = samples['t0']
X_t, y_t = [], []
for i in tqdm(range(len(obs))):
    X_t.append(np.concatenate((obs[i,0,:28], obs[i,0,30:31], controller[i], [(t0[i]%20)/20])))
    y_t.append(obs[i][-1][30]>0.13)

100%|██████████| 22006/22006 [00:00<00:00, 196000.67it/s]


In [166]:
opitmizer = ['lbfgs', 'sgd', 'adam'][2]
clf = MLPClassifier(solver=opitmizer, alpha=1e-5,hidden_layer_sizes=(50,50), verbose=True)

In [167]:
clf.fit(X, y) 

Iteration 1, loss = 0.61595610
Iteration 2, loss = 0.52651492
Iteration 3, loss = 0.45208965
Iteration 4, loss = 0.42025503
Iteration 5, loss = 0.40718764
Iteration 6, loss = 0.39484627
Iteration 7, loss = 0.38784355
Iteration 8, loss = 0.37990830
Iteration 9, loss = 0.37389550
Iteration 10, loss = 0.36935346
Iteration 11, loss = 0.36412122
Iteration 12, loss = 0.35954774
Iteration 13, loss = 0.35558724
Iteration 14, loss = 0.35187834
Iteration 15, loss = 0.34815411
Iteration 16, loss = 0.34533471
Iteration 17, loss = 0.34244130
Iteration 18, loss = 0.33848096
Iteration 19, loss = 0.33426869
Iteration 20, loss = 0.33094135
Iteration 21, loss = 0.32739018
Iteration 22, loss = 0.32313631
Iteration 23, loss = 0.32051338
Iteration 24, loss = 0.31629527
Iteration 25, loss = 0.31355569
Iteration 26, loss = 0.31221019
Iteration 27, loss = 0.30840788
Iteration 28, loss = 0.30764387
Iteration 29, loss = 0.30414336
Iteration 30, loss = 0.30226525
Iteration 31, loss = 0.30019138
Iteration 32, los

/home/timothee/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(50, 50), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=True, warm_start=False)

In [168]:
pred = clf.predict(X)

train_CM = [[0,0],[0,0]]
for i in range(len(y)):
    t = y[i]
    p = pred[i]
    train_CM[int(p)][int(t)] += 1
print('Train')
analyse_CM(train_CM, True);

Train
TPR:  0.9167493570401571
TNR:  0.8964492120909734
ACC:  0.9067324559153904
Precision:  0.9008775944757318
FS:  0.9087441785771639


In [169]:
pred = clf.predict(X_t)

eval_CM = [[0,0],[0,0]]
for i in range(len(y_t)):
    t = y_t[i]
    p = pred[i]
    eval_CM[int(p)][int(t)] += 1
print('eval')
analyse_CM(eval_CM, True);

eval
TPR:  0.8776213731686298
TNR:  0.8584277436651389
ACC:  0.8675361265109516
Precision:  0.8484539900018515
FS:  0.8627912449988232
